# REDCap App Log/Project Log Reshaping
This notebook reshapes the REDCap Mobile App Project Logs into Import Template format. It applies to situations when a project has multiple Data Collectors (RAs) in the field. For a number of them, both the "Send Data to Server" and "Send Emergency Data Dump" fail, but "Send Project Log" succeeds. This file parses the logs, reshapes them into the format of the import template, and combines them.

The code is designed for multi-wave projects. Feel free to adapt it for single-wave projects.

## Import Libraries

In [ ]:
from os.path import exists
import pandas as pd

## Extract data from app log

In [ ]:
def parser(df, n, template, RA, erroneous_column):
  '''
  There are three type of records in the App Log. 
  One type has both the "onload_val" and the "modified_val" variables. 
  The second type has only the "modified_val" variable. 
  The third type has neither.
  This parser can parse the first two types, and it is called by the extractor function below.
  When reading the import template, an erroneous column is usually introduced
  ex. parser(df_change, 1, template, RA, erroneous_column)
  '''
  # parse the 'Data' column by the field
  df_parsed = df.Data.str.split(r'\"\|\"',expand=True)
  # catch exception
  if df_parsed.isnull().values.any():
    print("There was an error in the parsing.")
    return None
  else:
    # add parsed data back
    df = pd.concat([df, df_parsed], axis=1)
    # extract record id
    df_record_id = df[1].str.split(r'\"\:\"',expand=True).rename(columns = {0:'record_id_column_name', 1:'record_id'})
    # add record id back
    df = pd.concat([df, df_record_id], axis=1)
    # extract wave number
    df_event_name = df[2].str.split(r'\"\:\"',expand=True).rename(columns = {0:'event_name_column_name', 1:'redcap_event_name'})
    # add wave number back
    df = pd.concat([df, df_event_name], axis=1)
    # extract field name
    df_field_name = df[4].str.split(r'\"\:\"',expand=True).rename(columns = {0:'field_name_column_name', 1:'field_name'})
    # add field name back
    df = pd.concat([df, df_field_name], axis=1)
    # n==1 means the rows have both "onload_val" and "modified_val" variables
    if n==1:
      # extract modified value from df_change
      df_modified_val = df[6].str.split(r'\"\:\"',expand=True).rename(columns = {0:'modified_val_col_name', 1:'modified_val'})
    # n==2 means the rows have only "modified_val" variable
    elif n==2:
      # extract modified value from df_no_change
      df_modified_val = df[5].str.split(r'\"\:\"',expand=True).rename(columns = {0:'modified_val_col_name', 1:'modified_val'})
    # add modified value back
    df = pd.concat([df, df_modified_val], axis=1)
    # keep the latest change for a field
    df = df.sort_values('Time').drop_duplicates(['record_id','redcap_event_name','field_name'],keep='last').reset_index(drop=True)
    # join two field - prepare to pivot
    cols=['record_id', 'redcap_event_name']
    df['record_wave']=df[cols].apply(lambda row: ':'.join(row.values.astype(str)), axis=1)
    # pivot
    df = df.pivot(index='record_wave', columns='field_name', values='modified_val')
    # remove the name of the column labels
    df.columns.name=None
    # reset index
    df=df.reset_index()
    # restore "record_id" and "redcap_event_name" columns
    df_record_event = df['record_wave'].str.split(r":", expand=True).rename(columns = {0:'record_id', 1:'redcap_event_name'})
    # add them back
    df = pd.concat([df, df_record_event], axis=1)
    # drop irrelevant columns
    df.drop('record_wave', inplace=True, axis=1)
    # drop rows with erroneous data
    df=df[df['redcap_event_name']=='wave2_arm_1'].reset_index(drop=True)
    # column names from df
    ls=df.columns.to_list()

    # read the import template
    df_template=pd.read_csv(template)
    # drop the erroneous column
    df_template.drop(erroneous_column, inplace=True, axis=1)
    # column names from template
    ls_template=df_template.columns.to_list()

    # find the columns that are in the data but not the template
    ls_diff = list(set(ls).difference(ls_template))

    # create the dataframe of columns to check by hand
    ls_manual = ls_diff.copy()
    ls_manual.append('record_id')
    ls_manual.append('redcap_event_name')
    df_manual=df[ls_manual]
    df_manual.to_csv(RA + '_' + str(n) + '_manual.csv', index=False)

    # merge the data into the template
    df_template=pd.concat([df_template, df], ignore_index=True, sort=False)
    # drop the extra columns
    df_template.drop(ls_diff, inplace=True, axis=1)
    df_template.to_csv(RA + '_' + str(n) + '_auto.csv', index=False)

In [ ]:
def extract_from_log(log, template, RA, erroneous_column):
  '''
  This function extracts data from log and saves the data in the format of an Import Template
  ex. extract_from_log('1668032732868.csv', 'ImportTemplate.csv', 'John', 'Unnamed: 1416')
  '''
  df_log = pd.read_csv(log)
  # keep the rows that have the action "MODIFY"
  df_log = df_log[df_log['Action']=='MODIFY'].reset_index(drop=True)
  # these records have the onload_val column
  df_change = df_log[df_log['Data'].str.contains("onload_val")].reset_index(drop=True)
  # these records do not have the onload_val column but have the modified_val column
  df_no_change = df_log[~df_log['Data'].str.contains("onload_val")].reset_index(drop=True)
  df_no_change = df_no_change[df_no_change['Data'].str.contains("modified_val")].reset_index(drop=True)
  # these records do not have the modified_val column
  df_no_modify = df_log[~df_log['Data'].str.contains("modified_val")].reset_index(drop=True)
  # parse the data
  parser(df_change, 1, template, RA)
  parser(df_no_change, 2, template, RA)
  if not df_no_modify.empty:
    df_parsed = df_no_modify.Data.str.split(r'\"\|\"',expand=True)
    # extract record id
    df_record_id = df_parsed[1].str.split(r'\"\:\"',expand=True).rename(columns = {0:'record_id_column_name', 1:'record_id'})
    df_no_modify.to_csv(RA + '_' + str(3) + '_manual.csv')
    df_record_id.to_csv(RA + '_' + str(3) + '_record_id.csv')

## Compare record ids
This section documents the records that are changed in the app log and compares them with the record ids that are given by the Project Manager (collected from the RAs)

In [ ]:
# A list of RAs
ls_names = ['Amy', 'Blake', 'Catherine', 'Dan', 'Elise', 'Frank', 'George']

In [ ]:
def extract_record_ids(type, ls_names):
  '''
  ex. extract_record_ids('1_auto', ls_names)
  '''
  for i, name in enumerate(ls_names):
    path = name + '_' + type + '.csv'
    if exists(path):
      df = pd.read_csv(name + '_' + type + '.csv')
      if i==0:
        ls_record_ids = df['record_id'].to_list()
      else:
        ls_record_ids = ls_record_ids + df['record_id'].to_list()
  return ls_record_ids

In [ ]:
ls_1_auto = extract_record_ids('1_auto', ls_names)
ls_1_manual = extract_record_ids('1_manual', ls_names)
ls_2_auto = extract_record_ids('2_auto', ls_names)
ls_2_manual = extract_record_ids('2_manual', ls_names)
ls_3_record_id = extract_record_ids('3_record_id', ls_names)

In [ ]:
ls_record_ids = ls_1_auto + ls_1_manual + ls_2_auto + ls_2_manual + ls_3_record_id

In [ ]:
# remove duplicates
ls_record_ids = [*set(ls_record_ids)]

In [ ]:
len(ls_record_ids)

71

In [ ]:
# import record ids obtained from the Project Manager
df_2 = pd.read_csv('record_ids.csv')
df_2

In [ ]:
ls_record_ids_2 = df_2['record_id'].to_list()
ls_record_ids_2

In [ ]:
# records that are in the Project Manager's list but not yours
ls_not_in = list(set(ls_record_ids_2) - set(ls_record_ids))
ls_not_in.sort()
ls_not_in

In [ ]:
# records that are in your list but not the Project Manager's
ls_should_not_be_in = list(set(ls_record_ids) - set(ls_record_ids_2))
ls_should_not_be_in.sort()
ls_should_not_be_in

In [ ]:
len(ls_should_not_be_in)

29

## Combine records

In [ ]:
def combine_records(type, ls_name):
  '''
  Combine records between RAs
  ex. combine_records('1_auto')
  '''
  for i, name in enumerate(ls_names):
    path = name + '_' + type + '.csv'
    if exists(path):
      if i == 0:
        df = pd.read_csv(path)
      else:
        df_temp = pd.read_csv(path)
        df = pd.concat([df, df_temp]).reset_index(drop=True)
  return df

### Running the code
This subsection runs the code that combines the records

In [ ]:
df_1_auto = combine_records('1_auto', ls_names)
df_2_auto = combine_records('2_auto', ls_names)
df_1_manual = combine_records('1_manual', ls_names)
df_2_manual = combine_records('2_manual', ls_names)
df_3_manual = combine_records('3_manual', ls_names)

In [ ]:
df_1_auto.to_csv('1_auto.csv',index=False)
df_2_auto.to_csv('2_auto.csv',index=False)
df_1_manual.to_csv('1_manual.csv',index=False)
df_2_manual.to_csv('2_manual.csv',index=False)
df_3_manual.to_csv('3_manual.csv',index=False)
# 1_auto.csv and 2_auto.csv are both for uploading to the project.
# 1_manual.csv and 2_manual.csv can contain calculated fields and/or hidden fields whose values have changed
# 3_manual.csv contains calculated fields and/or hidden fields with no value